In [1]:
# STEP 1: Installing required libraries 
!pip install flask pyngrok twilio --quiet

In [2]:
# STEP 2: Setting up SQLite database
import sqlite3
from datetime import datetime

conn = sqlite3.connect("expenses.db", check_same_thread=False)
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS expenses (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user TEXT,
    amount INTEGER,
    category TEXT,
    vendor TEXT,
    description TEXT,
    date TEXT
)
''')
conn.commit()
print(" SQLite database and 'expenses' table ready.")

 SQLite database and 'expenses' table ready.


In [3]:
# STEP 3: Defining a function to save expenses
import sqlite3

def save_expense(user, amount, category, vendor, description, date):
    conn = sqlite3.connect("expenses.db")  # Create a new connection inside the function
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO expenses (user, amount, category, vendor, description, date)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (user, amount, category, vendor, description, date))
    conn.commit()
    conn.close()  # Close the connection after each use


In [4]:
import re

In [5]:
def detect_category(msg):
    msg = msg.lower()

    category_keywords = {
        "food": ["tea", "coffee", "pizza", "biryani", "snacks", "breakfast", "lunch", "dinner", "swiggy", "zomato", "ccd", "starbucks","pizza"],
        "essentials": ["groceries", "milk", "vegetables", "zepto", "blinkit", "amazon pantry"],
        "medicines": ["pharmacy", "tablet", "medicine", "1mg", "medlife", "prescription", "chemist"],
        "sports": ["gym", "shoes", "football", "cricket", "bat", "sports", "yoga", "exercise","jersey"]
    }

    for category, keywords in category_keywords.items():
        if any(word in msg for word in keywords):
            return category

    return "other"  # fallback


In [6]:
def get_spending_between_dates(user, start_date, end_date, category=None):
    if category:
        cursor.execute("""
            SELECT SUM(amount) FROM expenses
            WHERE user = ? AND category = ? AND date BETWEEN ? AND ?
        """, (user, category, start_date, end_date))
    else:
        cursor.execute("""
            SELECT SUM(amount) FROM expenses
            WHERE user = ? AND date BETWEEN ? AND ?
        """, (user, start_date, end_date))
    result = cursor.fetchone()[0]
    return result if result else 0



In [7]:
def correct_expense(user, old_amount, new_amount, date):
    import sqlite3

    conn = sqlite3.connect("expenses.db")
    cursor = conn.cursor()

    try:
        # First: find the latest matching row's ID
        cursor.execute("""
            SELECT id FROM expenses
            WHERE user = ? AND amount = ? AND date(date) = ?
            ORDER BY id DESC
            LIMIT 1
        """, (user, old_amount, date.strftime("%Y-%m-%d")))
        result = cursor.fetchone()

        if result:
            expense_id = result[0]
            cursor.execute("""
                UPDATE expenses
                SET amount = ?
                WHERE id = ?
            """, (new_amount, expense_id))
            conn.commit()
            return True
        else:
            return False

    except Exception as e:
        print("Correction error:", e)
        return False
    finally:
        conn.close()


In [8]:
from flask import Flask, request
from twilio.twiml.messaging_response import MessagingResponse

app=Flask(__name__)
known_users = set()  # This should be declared globally

@app.route("/webhook", methods=["POST"])
def webhook():
    import re
    from datetime import datetime

    incoming_msg = request.form.get('Body') or ""
    sender = request.form.get('From') or ""
    msg_lower = incoming_msg.lower()

    print(" Incoming:", incoming_msg)

    response = MessagingResponse()
    msg = response.message()

    # 👋 Welcome message
    if sender not in known_users:
        known_users.add(sender)
        msg.body("👋 Welcome! You can start by saying something like 'Spent 200 on pizza'. I'll track your expenses and answer your queries.")
        return str(response)
    # STEP: Handle correction of expenses
    if re.search(r"(change|correct|update|not)\s.*(\d+)", msg_lower):
        # Pattern 1: "change 100 to 200", "update 100 to 200"
        match1 = re.search(r"(?:change|correct|update)\s+(?:rs|₹)?\s?(\d+)\s+(?:to)\s+(?:rs|₹)?\s?(\d+)", msg_lower)

        # Pattern 2: "200 not 100", "update 200 not 100"
        match2 = re.search(r"(?:update|change|correct)?\s*(?:rs|₹)?\s?(\d+)\s+(?:not|instead of)\s+(?:rs|₹)?\s?(\d+)", msg_lower)

        # Date pattern: flexible
        date_match = re.search(r"(\d{1,2}\s+\w+\s+\d{4}|\d{1,2}/\d{1,2}/\d{2,4})", msg_lower)

        if match1:
            old_amount = int(match1.group(1))
            new_amount = int(match1.group(2))
        elif match2:
            new_amount = int(match2.group(1))
            old_amount = int(match2.group(2))
        else:
            msg.body("❌ Couldn't understand the correction. Try 'change 100 to 200' or '200 not 100'")
            return str(response)

        # Parse date if present
        expense_date = None
        if date_match:
            try:
                expense_date = datetime.strptime(date_match.group(1), "%d %B %Y")
            except:
                try:
                    expense_date = datetime.strptime(date_match.group(1), "%d/%m/%Y")
                except:
                    try:
                        expense_date = datetime.strptime(date_match.group(1), "%d/%m/%y")
                    except:
                        pass
        if not expense_date:
            expense_date = datetime.now().date()

        # ✅ Correct the record using the function
        success = correct_expense(sender, old_amount, new_amount, expense_date)
        if success:
            msg.body(f"✅ Updated your record: {old_amount} → {new_amount} on {expense_date.strftime('%d %b %Y')}")
        else:
            msg.body("⚠️ Couldn't find a matching expense to correct.")
        return str(response)


            
    # 💸 STEP 1: Handle expense entries
    elif any(word in msg_lower for word in ["spent", "paid", "bought"]):
        amount_match = re.findall(r'(?:₹|rs)?\s?(\d+)', msg_lower)
        if amount_match:
            amount = int(amount_match[0])
            category = detect_category(incoming_msg)
            vendor = "unknown"
            description = incoming_msg
            date = datetime.now().strftime("%Y-%m-%d")

            save_expense(sender, amount, category, vendor, description, date)
            msg.body(f"💰 Got it! You've spent ₹{amount} on {category}.")
        else:
            msg.body("❌ Couldn't find the amount. Try: 'Spent ₹300 on snacks'.")

    # 📅 STEP 2: Handle total queries with time range
    elif "from" in msg_lower and "to" in msg_lower:
        date_match = re.findall(r"\d{4}-\d{2}-\d{2}", msg_lower)
        categories = ["food", "essentials", "medicines", "sports"]
        found_category = None
        for cat in categories:
            if cat in msg_lower:
                found_category = cat
                break

        if len(date_match) == 2:
            start_date, end_date = date_match
            total = get_spending_between_dates(sender, start_date, end_date, found_category)
            if found_category:
                msg.body(f"📊 You spent ₹{total} on {found_category} from {start_date} to {end_date}.")
            else:
                msg.body(f"📊 You spent ₹{total} from {start_date} to {end_date}.")
        else:
            msg.body("⚠️ Please use date format: from YYYY-MM-DD to YYYY-MM-DD")

    # 🧾 STEP 3: Handle category-wise or total queries
    elif re.search(r"(how much|what.*spend|total.*spend|spent.*total|how much have i spent)", msg_lower):
        print(" Matched as query:", msg_lower)
        categories = ["food", "essentials", "medicines", "sports"]
        found_category = None

        for cat in categories:
            if cat in msg_lower:
                found_category = cat
                break

        if found_category:
            cursor.execute("SELECT SUM(amount) FROM expenses WHERE user = ? AND category = ?", (sender, found_category))
            total = cursor.fetchone()[0] or 0
            msg.body(f"📦 You’ve spent ₹{total} on {found_category}.")
        else:
            cursor.execute("SELECT SUM(amount) FROM expenses WHERE user = ?", (sender,))
            total = cursor.fetchone()[0] or 0
            msg.body(f"💸 You’ve spent ₹{total} in total.")

    

    # ❓ STEP 4: Unknown message
    else:
        msg.body("🤖 Try: 'Spent ₹300 on coffee', 'How much on food?', or 'How much from 2025-05-01 to 2025-05-10?'")

    return str(response)




In [9]:
# STEP 5: Running Flask server in background
import threading

def run_flask():
    app.run(host='0.0.0.0', port=5000)

thread = threading.Thread(target=run_flask)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.3:5000
Press CTRL+C to quit
127.0.0.1 - - [29/May/2025 14:06:36] "POST /webhook HTTP/1.1" 200 -


 Incoming: Hi


127.0.0.1 - - [29/May/2025 14:06:41] "POST /webhook HTTP/1.1" 200 -


 Incoming: Spent 400 on lunch


...

 Incoming: how much on food?
 Matched as query: how much on food?


127.0.0.1 - - [29/May/2025 14:07:58] "POST /webhook HTTP/1.1" 200 -


 Incoming: change 409 to 490 on 29 may 2025


[2025-05-29 14:17:45,479] ERROR in app: Exception on /webhook [POST]
Traceback (most recent call last):
  File "C:\Users\varad\AppData\Roaming\Python\Python310\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\varad\AppData\Roaming\Python\Python310\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\varad\AppData\Roaming\Python\Python310\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\varad\AppData\Roaming\Python\Python310\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "C:\Users\varad\AppData\Local\Temp\ipykernel_102412\504955289.py", line 124, in webhook
    cursor.execute("SELECT SUM(amount) FROM expenses WHERE user = ?", (sender,))
sqlite3.ProgrammingError: SQLite 

 Incoming: how much on medicine
 Matched as query: how much on medicine


In [10]:

# STEP 6: Starts ngrok and we will get public URL
from pyngrok import ngrok

ngrok.kill()  # kill old sessions if any
public_url = ngrok.connect(5000)
print(f"🌐 Public webhook URL: {public_url}/webhook")


🌐 Public webhook URL: NgrokTunnel: "https://b2ac-2401-4900-1cb9-aea-b910-8b1e-87f4-b3ba.ngrok-free.app" -> "http://localhost:5000"/webhook


In [11]:
import pandas as pd
import sqlite3

# Connect to your database
conn = sqlite3.connect("expenses.db")

# Load the expenses table into a DataFrame
df = pd.read_sql_query("SELECT * FROM expenses", conn)

# Show the first few rows
df.head()


,id,user,amount,category,vendor,description,date
0,1,whatsapp:+919148968076,500,food,unknown,Spent 500 on dinner,2025-05-28
1,2,whatsapp:+919148968076,200,essentials,unknown,Spent 200 on milk,2025-05-28
2,3,whatsapp:+919148968076,5000,medicines,unknown,Spent 5000 on medicines,2025-05-28
3,4,whatsapp:+919148968076,200,food,unknown,Spent 200 on lunch,2025-05-28
4,5,whatsapp:+919148968076,500,food,unknown,Spent 500 on dinner,2025-05-28


In [12]:
df.to_csv("expenses_export.csv", index=False)
print("Exported to expenses_export.csv")


Exported to expenses_export.csv


In [13]:
import sqlite3

conn = sqlite3.connect("expenses.db")
cursor = conn.cursor()

cursor.execute("PRAGMA table_info(expenses)")
columns = cursor.fetchall()

for col in columns:
    print(col)

conn.close()


(0, 'id', 'INTEGER', 0, None, 1)
(1, 'user', 'TEXT', 0, None, 0)
(2, 'amount', 'INTEGER', 0, None, 0)
(3, 'category', 'TEXT', 0, None, 0)
(4, 'vendor', 'TEXT', 0, None, 0)
(5, 'description', 'TEXT', 0, None, 0)
(6, 'date', 'TEXT', 0, None, 0)
